In [48]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [49]:
#Criando uma session com acesso ao gcp
spark = (
    SparkSession
    .builder
    .appName("Desafio-edc-format-refined")
    .config("spark.serializer","org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)

In [32]:
""" .config("spark.jars","./jars/gcs-connector-hadoop3-latest.jar")
    .config("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
    .config("fs.AbstractFileSystem.gs.impl","com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
    .config("fs.gs.auth.service.account.enable","true")
    .config("fs.gs.auth.service.account.json.keyfile", "/mnt/d/EngDados/gcp/gcp-estudos-engdados-20f2cdfffed8.json")
    .config("spark.driver.memory", "8g")
"""

' .config("spark.jars","./jars/gcs-connector-hadoop3-latest.jar")\n    .config("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\n    .config("fs.AbstractFileSystem.gs.impl","com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\n    .config("fs.gs.auth.service.account.enable","true")\n    .config("fs.gs.auth.service.account.json.keyfile", "/mnt/d/EngDados/gcp/gcp-estudos-engdados-20f2cdfffed8.json")\n    .config("spark.driver.memory", "8g")\n'

## Junção dos dados de estabelecimento com municipio e atividade economica


In [60]:
df_estabelecimento = spark.read.parquet("gs://bootcamp-edc/trusted/estabelecimento.parquet/*.parquet")

In [61]:
df_cnae = spark.read.parquet("gs://bootcamp-edc/trusted/cnae.parquet/*.parquet")

In [55]:
df_estabelecimento.printSchema()

root
 |-- cnpj_basico: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dig_verificador: string (nullable = true)
 |-- idc_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- cod_situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: string (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- cod_pais: integer (nullable = true)
 |-- data_inicio_atividade: string (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: integer (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- nome_logradouro: string (nullable = true)
 |-- numero_logradouro: string (nullable = true)
 |-- complemento_logradouro: string (nullable = true)
 |-- nome_bairro: string (nullable = true)
 |-- cod_cep: string (nullable = true)
 |-- sigla_uf: string (nullable = true)
 |-- cod_municipio: i

In [36]:
df_cnae.printSchema()

root
 |-- cod_atividade: integer (nullable = true)
 |-- desc_atividade: string (nullable = true)



In [62]:
df_estabelecimento_enriquecido = df_estabelecimento.join(df_cnae,f.col("cnae_fiscal_principal") == f.col("cod_atividade"),"left")

In [63]:
df_estabelecimento_enriquecido = df_estabelecimento_enriquecido.join(df_cnae.withColumnRenamed("desc_atividade", "desc_atividade_secundaria")
                                                                     .withColumnRenamed("cod_atividade", "cod_atividade_secundaria"),
                                                                     f.col("cnae_fiscal_secundaria") == f.col("cod_atividade_secundaria"),"left")

In [64]:
df_estabelecimento_enriquecido.limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dig_verificador,idc_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,cod_pais,...,num_telefone_2,ddd_fax,num_fax_2,email,situacao_especial,data_situacao_especial,cod_atividade,desc_atividade,cod_atividade_secundaria,desc_atividade_secundaria
0,02590637,0001,00,1,Jw Empreiteira,8,20081231,71,None,NaN,...,None,None,None,None,None,None,4120400,Construção de edifícios,4299501.0,Construção de instalações esportivas e recreat...
1,05980213,0001,40,1,Residencial Ribeira Ii,2,20030908,0,None,NaN,...,None,None,None,None,None,None,8112500,Condomínios prediais,NaN,None
2,00963122,0001,83,1,Bar E Mercearia Victoria,8,20000905,1,None,NaN,...,None,None,None,None,None,None,5611204,Bares e outros estabelecimentos especializados...,NaN,None
3,05980277,0001,41,1,Eletro Cell Celulares,4,20180828,63,None,NaN,...,None,None,None,None,None,None,4752100,Comércio varejista especializado de equipament...,NaN,None
4,04313498,0001,01,1,Laffer Industria E Comercio De Confeccoes,2,20051103,0,None,NaN,...,None,None,None,None,None,None,1412601,"Confecção de peças de vestuário, exceto roupas...",1412603.0,"Facção de peças do vestuário, exceto roupas ín..."


In [65]:
df_municipio = spark.read.parquet("gs://bootcamp-edc/trusted/municipio.parquet/*.parquet")

In [40]:
df_municipio.printSchema()

root
 |-- cod_municipio: integer (nullable = true)
 |-- desc_municipio: string (nullable = true)



In [66]:
df_estabelecimento_enriquecido = df_estabelecimento_enriquecido.join(df_municipio, "cod_municipio", "left")

In [58]:
df_estabelecimento_enriquecido.limit(5).toPandas()

,cod_municipio,cnpj_basico,cnpj_ordem,cnpj_dig_verificador,idc_matriz_filial,nome_fantasia,cod_situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,...,ddd_telefone_2,num_telefone_2,ddd_fax,num_fax_2,email,situacao_especial,data_situacao_especial,cod_atividade,desc_atividade_secundaria,desc_municipio
0,9643,02590637,0001,00,1,Jw Empreiteira,8,20081231,71,None,...,None,None,None,None,None,None,None,4299501.0,Construção de instalações esportivas e recreat...,Xambioa
1,1761,05980213,0001,40,1,Residencial Ribeira Ii,2,20030908,0,None,...,None,None,None,None,None,None,None,NaN,None,Natal
2,6221,00963122,0001,83,1,Bar E Mercearia Victoria,8,20000905,1,None,...,None,None,None,None,None,None,None,NaN,None,Bebedouro
3,9373,05980277,0001,41,1,Eletro Cell Celulares,4,20180828,63,None,...,None,None,None,None,None,None,None,NaN,None,Goiania
4,9431,04313498,0001,01,1,Laffer Industria E Comercio De Confeccoes,2,20051103,0,None,...,None,None,None,None,None,None,None,1412603.0,"Facção de peças do vestuário, exceto roupas ín...",Jaragua


In [67]:
df_estabelecimento_enriquecido.printSchema()

root
 |-- cod_municipio: integer (nullable = true)
 |-- cnpj_basico: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dig_verificador: string (nullable = true)
 |-- idc_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- cod_situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: string (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- cod_pais: integer (nullable = true)
 |-- data_inicio_atividade: string (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: integer (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- nome_logradouro: string (nullable = true)
 |-- numero_logradouro: string (nullable = true)
 |-- complemento_logradouro: string (nullable = true)
 |-- nome_bairro: string (nullable = true)
 |-- cod_cep: string (nullable = true)
 |-- sigla_uf: 

In [68]:
df_estabelecimento_enriquecido.write.mode("overwrite").parquet("gs://bootcamp-edc/refined/estabelecimento.parquet")

In [69]:
del(df_estabelecimento)

In [70]:
spark.stop()